In [23]:
import pdfplumber

def extract_text_with_spaces(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = ''
            words = []
            previous_char = None

            # Extract character-level details
            for char in page.chars:
                if previous_char:
                    # Calculate space based on character positions
                    space_threshold = 0.5 * (char['fontname'] == previous_char['fontname']) # Adjust this threshold if needed
                    if char['x0'] > previous_char['x1'] + space_threshold:
                        page_text += ' '
                page_text += char['text']
                previous_char = char

            text += page_text + '\n'
    return text

# Use the function to extract text with spaces
pdf_path = 'C:\\MyFolder\\earningscall.pdf'
text = extract_text_with_spaces(pdf_path)

# Print or process the text as needed
print(text)


Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures.Reconciliations to the most directly comparable GAAP financial measures are provided in our shareholderletter on our Inves

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example transcript
transcript = text

# Create the text splitter with desired parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Maximum size of each chunk in characters
    chunk_overlap=500,  # Overlap between chunks to preserve context
    separators=["\n\n", "\n", ".", " "]  # Order of separators to use for splitting
)

# Split the transcript into chunks
chunks = text_splitter.split_text(transcript)

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")


Chunk 1:
Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures

Chunk 2:
. Before we get started, I'd like to remind you that during today's call we maymake forward-looking sta

In [33]:
from langchain_core.documents import Document

# Convert chunks to documents
documents = [Document(page_content=chunk, metadata={"source": "transcript"}) for chunk in chunks]


In [38]:
# Output the documents
for i, doc in enumerate(documents):
    print(doc)

page_content='Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures' metadata={'source': 'transcript'}
page_content='. Before we get started, I'd like to remind you that during

In [39]:
from langchain.llms import OpenAI
import chromadb
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Assuming 'chunks' is a list of strings, each representing a chunk of the transcript

# Convert chunks to the format expected by the vectorstore
# Chroma expects each document to be an object with a 'page_content' attribute.

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="nomic-ai/nomic-embed-text-v1.5-GGUF", check_embedding_ctx_length=False
)

# Initialize the ChromaDB client (using an ephemeral in-memory client for testing)
new_client = chromadb.EphemeralClient()

# Create the vectorstore and insert the documents
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, client=new_client)
print("Finished creating the vectorstore.")

# Initialize the retriever for use in the RAG pipeline
retriever = vectorstore.as_retriever()


Finished creating the vectorstore.


In [44]:
# Retrieve the documents stored in the vectorstore
stored_docs = vectorstore._collection.get(include=["documents", "metadatas"])

# Display the contents of each document
for i, doc in enumerate(stored_docs['documents']):
    print(f"Document {i+1}:")
    print(f"Content: {doc}")
    print(f"Metadata: {stored_docs['metadatas'][i]}")
    print("=" * 50)  # Separator for readability


Document 1:
Content: . We do it the compliant way, the secure, trusted way. And so we're the trusted counterparty thatmany of these folks have been waiting for to enter the market. And I think that's going to pay off as a reallygood long-term strategy. Alesia, anything you want to add?Alesia Haas, CFO : The only thing that I would add, as Brian said, it is early. And at this point in time,derivatives, while an important future growth driver, is not a material driver of our financial results.What I wanted to comment on in Q2, though, is because we do report revenue associated with derivatives butnot volume, what you can see is a beginning disconnect between the revenue and the volume that we report,which is spot.And so the growth rates of those two are the change, you need to understand that there is a disconnectbetween numerator and denominator when you may calculate blended average fees for our platform. Thatwas the important message that I wanted to communicate in the quarter
Metadat

In [45]:
# Retrieve the embeddings stored in the vectorstore
stored_embeddings = vectorstore._collection.get(include=["embeddings"])

# Display the embeddings
for i, embedding in enumerate(stored_embeddings['embeddings']):
    print(f"Embedding {i+1}:")
    print(embedding)
    print("=" * 50)  # Separator for readability


Embedding 1:
[0.0653366893529892, 0.07932306826114655, -0.18919159471988678, -0.00017052919429261237, 0.07058703899383545, -0.020244402810931206, 0.06748233735561371, -0.007384806871414185, -0.002709988271817565, -0.052068524062633514, 0.03475845605134964, 0.02207823097705841, 0.04061224311590195, 0.013488784432411194, 0.0745062455534935, 0.000665066356305033, -0.005957693327218294, -0.04925350099802017, 0.03159508854150772, 0.06263794004917145, -0.07225694507360458, -0.06574087589979172, -0.01780867762863636, -0.04110457003116608, 0.046271342784166336, 0.008123204112052917, -0.014275520108640194, -0.01077268272638321, 0.03910320624709129, 0.002070088405162096, 0.06472386419773102, 0.026727687567472458, 0.027027597650885582, -0.022686339914798737, -0.07349498569965363, -0.03369217738509178, -0.030642226338386536, 0.034734148532152176, 0.01173209398984909, -0.08493762463331223, 0.05363820865750313, 0.017797112464904785, -0.043950796127319336, -0.04235301539301872, -0.003293678630143404,